In [1]:
%reload_ext sql
import sys
reload(sys)
sys.setdefaultencoding("utf-8")

In [2]:
%sql postgresql://postgres@centos72/postgres
%sql select 1;
%sql select madlib.version();

version
"MADlib version: 1.10.0-dev, git revision: rel/v1.9.1-8-g82e56a4, cmake configuration time: 2016年 11月 22日 火曜日 00:25:23 UTC, build type: RelWithDebInfo, build system: Linux-3.10.0-327.el7.x86_64, C compiler: gcc 4.8.5, C++ compiler: g++ 4.8.5"


In [39]:
%%sql

drop table if exists window_test;
create table window_test(
     test_date date
    ,shop_id int
    ,value int);
insert into window_test values 
 ('2016-12-1', '1', '3')
,('2016-12-1', '2', '2')
,('2016-12-1', '1', '5')
,('2016-12-2', '1', '4')
,('2016-12-3', '1', '2')
,('2016-12-3', '2', '6')
,('2016-12-3', '2', '1')
,('2016-12-4', '1', '3')
,('2016-12-4', '1', '4');
select * from window_test;

test_date,shop_id,value
2016-12-01,1,3
2016-12-01,2,2
2016-12-01,1,5
2016-12-02,1,4
2016-12-03,1,2
2016-12-03,2,6
2016-12-03,2,1
2016-12-04,1,3
2016-12-04,1,4


In [56]:
%%sql

select test_date, shop_id, avg(value) 
from window_test
group by shop_id, test_date
order by test_date, shop_id;

test_date,shop_id,avg
2016-12-01,1,4.0000000000000000
2016-12-01,2,2.0000000000000000
2016-12-02,1,4.0000000000000000
2016-12-03,1,2.0000000000000000
2016-12-03,2,3.5000000000000000
2016-12-04,1,3.5000000000000000


In [57]:
%%sql

select test_date, shop_id, 
    avg(value) over (
    partition by shop_id, test_date)
from window_test
order by test_date, shop_id;

test_date,shop_id,avg
2016-12-01,1,4.0000000000000000
2016-12-01,1,4.0000000000000000
2016-12-01,2,2.0000000000000000
2016-12-02,1,4.0000000000000000
2016-12-03,1,2.0000000000000000
2016-12-03,2,3.5000000000000000
2016-12-03,2,3.5000000000000000
2016-12-04,1,3.5000000000000000
2016-12-04,1,3.5000000000000000


In [60]:
%%sql

select test_date, shop_id, value, 
    rank() over (
    partition by test_date
    order by value)
from window_test
order by test_date, rank;

test_date,shop_id,value,rank
2016-12-01,2,2,1
2016-12-01,1,3,2
2016-12-01,1,5,3
2016-12-02,1,4,1
2016-12-03,2,1,1
2016-12-03,1,2,2
2016-12-03,2,6,3
2016-12-04,1,3,1
2016-12-04,1,4,2
